# Advanced Dataframes Exercises I

### 1. Run python -m pip install mysqlclient pymysql from your terminal to install the mysql client (any folder is fine)

### 2. cd into your exercises folder for this module and run echo env.py >> .gitignore

### 3. Create a function named `get_db_url`. It should accept a username, hostname, password, and database name and return a url connection string formatted like in the example at the start of this lesson.

In [14]:
from env import user, password, host
import pandas as pd
import numpy as np

def get_db_url(username: str, hostname: str , password: str, database_name: str):
    '''
    Takes username, hostname, password and database_name and 
    returns a connection string
    '''
    connection = f'mysql+pymysql://{username}:{password}@{hostname}/{database_name}'
    
    return connection

conn = get_db_url(user, host, password, 'employees')


### 4. Use your function to obtain a connection to the employees database.

In [7]:
sql = '''
select *
from employees
limit 5
offset 50
'''
pd.read_sql(sql, conn)

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10051,1953-07-28,Hidefumi,Caine,M,1992-10-15
1,10052,1961-02-26,Heping,Nitsch,M,1988-05-21
2,10053,1954-09-13,Sanjiv,Zschoche,F,1986-02-04
3,10054,1957-04-04,Mayumi,Schueller,M,1995-03-13
4,10055,1956-06-06,Georgy,Dredge,M,1992-04-27


### 5a. Intentionally make a typo in the database url. What kind of error message do you see? -- Returns a NoSuchModuleError

In [9]:
bad_conn = conn[:5] + 'test' + conn[5:]
pd.read_sql(sql, bad_conn)

NoSuchModuleError: Can't load plugin: sqlalchemy.dialects:mysqltest.pymysql

### 5b. Intentionally make an error in your SQL query. What does the error message look like? -- Returns a ProgrammingError which then returns the specific pymysql.err information

In [13]:
bad_sql = '''
select *
from employeeeeeeeeees
limit 5
offset 50
'''
pd.read_sql(bad_sql, conn)

ProgrammingError: (pymysql.err.ProgrammingError) (1146, "Table 'employees.employeeeeeeeeees' doesn't exist")
[SQL: 
select *
from employeeeeeeeeees
limit 5
offset 50
]
(Background on this error at: http://sqlalche.me/e/14/f405)

### 6. Read the employees and titles tables into two separate DataFrames.

In [24]:
sql = '''
select *
from employees
'''
employees_df = pd.read_sql(sql, conn)
sql = '''
select *
from titles
'''
titles_df = pd.read_sql(sql, conn)


(300024, 6)

### 7. How many rows and columns do you have in each DataFrame? Is that what you expected? -- Yes to both

In [26]:
employees_df.shape
titles_df.shape

(443308, 4)

### 8. Display the summary statistics for each DataFrame.

In [29]:
employees_df.describe()
titles_df.describe()

,emp_no
count,443308.000000
mean,253075.034430
std,161853.292613
min,10001.000000
25%,84855.750000
50%,249847.500000
75%,424891.250000
max,499999.000000


### 9. How many unique titles are in the `titles` DataFrame?

In [34]:
len(titles_df.title.unique())

7

### 10. What is the oldest date in the to_date column?

In [37]:
titles_df.to_date.min()

datetime.date(1985, 3, 1)

### 11. What is the most recent date in the to_date column?

In [59]:
titles_df[titles_df.to_date != datetime.date(9999, 1, 1)].to_date.max()

datetime.date(2002, 8, 1)